# Real-Time MRI with BART 🎥

## Structure

### Part 1: Simple Real-Time MRI Reconstruction

Using the adjoint NuFFT paired with the Ram-Lak Filter, a real-time dataset
acquired with a turn-based radial FLASH sequence is reconstructed.

### Part 2: Iterative Reconstruction

We transform the previous reconstruction into something more modern.

### Part 3: iGRASP and Real-Time NLINV

We use BART to run the well-known iGRASP method.

A previously "hidden" step, the creation of coil sensitivities, is examined further. We look at a different iterative reconstruction scheme which is very suitable for real-time MRI and real-time reconstruction.

## Setup Environment for BART

This notebook assumes bart is 'available', i.e. has been successfully installed beforehand.


If this doesn't work, please consult the [installation guide](https://mrirecon.github.io/bart/installation.html).

✅ You should be able to run bart "directly" in this notebook:

In [ ]:
! bart version

# Part 1: Simple Real-Time MRI Reconstruction

In this part, we will reconstruct a real-time MRI dataset recorded using a turn-based linear trajectory.

We create a new directory and go there first:

In [ ]:
! mkdir -p session3_1
%cd session3_1

## Acquiring the data

Download the file from TU Graz cloud if it is not already there. The file is about 70MB big, so it shouldn't take too long.

In [ ]:
%%bash
[ -f ksp_turns5.ra ] || wget https://cloud.tugraz.at/index.php/s/N79NSwJSFaBM7mY/download/ksp_turns5.ra
bart copy ksp_turns5.ra ksp

✏️ **Use the BART `show` tool to confirm the dimensions of the raw data.**

In [ ]:
!

<details>
    <summary>Show solution</summary>

    ! bart show -m ksp
</details>

✅ You should now have a file called "ksp" which contains 100 k-space frames.

There are 256 samples and the number of spokes is 13.

**The number of turns is 5.** This is, however, not visible from the "ksp" file.

We do a small preprocessing step which will be explained later:

In [ ]:
%%bash
bart reshape 7 1 256 13 ksp ksp

Followed by another preprocessing step as an exercise!

✏️ **Preprocessing**:
- Use the bart `cc` (coil compression) tool to reduce the number of channels in this file to 6!

Output filename "ksp_cc"

In [ ]:
%%bash



<details>
    <summary>Show solution</summary>

    %%bash

    bart cc -p6 ksp ksp_cc
</details>

✅ You should now have a file called "ksp_cc" which has dimension 6 instead of 26 in axis 3.

## Sliding window

The next step is to create the "sliding window", i.e., we will create a file which has 5 * 13 = 65 spokes and 100 frames.
To achieve this, for every output frame, the spokes of the next four frames (or 0, as available) should follow after the 13 spokes of the current input frame.

This tutorial is guiding you along one way to achieve this, but there are certainly others as well. :)

✏️ **Creating shifted copies of the input file**:
- `bart extract` can be used to copy only part of an array into a new file (i.e., cropping).
- `bart resize` can be used for cropping as well, but it can also create zero-padded versions of an array.

First, use bart extract to remove the first frame from the ksp file.

Repeat this four times, removing also the second, third, and fourth frame.

Zero-Pad the resulting arrays at the end, so that the resulting files have 100 frames again.

Output filenames: ksp_1, ksp_2, ksp_3

In [ ]:
%%bash
## create sliding window by joining shifted copies of the data
for i in $(seq 4); do

done


<details>
    <summary>Show solution</summary>

    %%bash
    ## create sliding window by joining shifted copies of the data
    for i in $(seq 4); do
        bart extract 10 $i 100 ksp_cc ksp_$i
        bart resize 10 100 ksp_$i ksp_$i
    done

</details>

✅ You should now have four files which have the same shape as the ksp_cc file, but are shifted and zero-padded (along dim 10) copies of it.

You can run the following code as a test to see if your result is correct:

In [ ]:
%%bash
bart slice 10 3 ksp_cc tmp1
bart slice 10 0 ksp_3 tmp2
bart nrmse -t0 tmp1 tmp2

In the next step, `join` the shifted copies AND the original file into a new array, using the bart command of the same name.

Let's say you join the files along dimension "11". The dimensionality of the resulting array is then (256,13,6,100,5) (without singleton dims).

Goal of this exercise was to get (65) instead of (13,5) - so we're not quite there yet.

Similar to numpy etc., there is a bart command "reshape" which works pretty similarly. However, there is a big pitfall here:

The turn-based pattern means that spokes from subsequent frames are interleaved.
We want the same for our resulting file, meaning, the first spoke in the result should be from the original, the second from the 1-shifted copy (aka the second frame), the third from the 2-shifted copy, and so on.

For reshaping, one needs to know that BART uses **Fortran** or **Column-Major** storage. That is, if an axis comes first, neighbouring indices there are close in memory.
Or in other words, think of a 2D matrix. Rows are the first index, columns the second. In BARTs file format, the columns are stored one after another, values that are neighbours in a column (top-down) are neighbours in memory.
(Numpy defaults to the opposite, i.e. rows are stored one after another.)

To come back to our array, if we were to reshape while the spoke axis is before the shifted-copies axis, the shifted copies are far away and we get the wrong behaviour. Thus, we need to transpose first!

✏️ **Re-Arranging the data**
- `join` the original array + the shifted copies along dimension 11 into a new array.
- use the `bart transpose` command to swap the spoke and "shift"-axes
- `reshape` to the desired shape.

Output file: ksp_sw

<details>
    <summary>Bitmasks AKA flags</summary>

A set of dimensions (e.g., 0,2,3) can be specified in multiple ways.

Which way needs to be used depends on the context in BART .

Some commands just expect the dimensions (e.g., bart extract), typically separated by additional arguments (index along that dimension).

Other commands take a so-called bitmask - that is, the number that would result if the bits specified by the given dimensions are set to 1.

So, the list of dimensions (0, 2, 3) becomes 1011b = 13.

This actually saves some typing and you only need a single number, not a list of numbers (which often makes things way easier if you are programming close to the hardware :).

There is a bart tool which saves you the effort of manually calculcating bitmasks:

`bart bitmask` takes a list of space-separated dimensions and outputs the correspdoning bitmask.

The reverse can be accomplished by using `bart bitmask -b`.

</details>

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>

    %%bash
    bart join 11 ksp_cc $(seq -s' ' -f 'ksp_%g' 4) tmp
    bart transpose 2 11 tmp tmp
    bart reshape $(bart bitmask 2 11) 65 1 tmp ksp_sw
</details>

✅ You should now have an output file with dimensions (256,65,1,6,1,...,1,100)

To check that it worked:

In [ ]:
%%bash

bart slice 2 1 10 0 ksp_sw tmp1
bart slice 2 0 10 1 ksp_cc tmp2
bart nrmse tmp1 tmp2

## Trajectory

Next, we need a trajectory that describes the (spatial) frequencies at which our k-space data was actually measured.

bart can create turn-based trajectories using the `-t` switch of the `traj` tool.

Furthermore, we need
- the `-r` for radial
- `-D` for double-angle
- `-x` and `-y`,
- `-o2`, because Siemens does default two-times oversampling.

Finding the specific command line for a given dataset can be a challenge, if no further information is available.

In [ ]:
%%bash

##Create trajectory
bart traj -r -D -o2 -x128 -y13 -t5 trj_turns

Because the turn-based trajectory repeats after a few frames, the output doesn't have nearly the same shape as the ksp file.

Using `repmat` (similar to np.tile), you can create an array which contains several copies of the input array.

With `reshape`, you can bring that file into the same shape as the original ksp data.

✏️ **Make the trajectory match the data**:
- Use repmat and reshape to create a trajectory with 100 frames.

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>
    
    %%bash
    bart repmat 11 20 trj_turns tmp
    bart reshape $(bart bitmask 10 11) 100 1 tmp trj
</details>

✅ You should now have an output file with dimensions (3, 256, 13, 1, ..., 1, 100)

Trajectories, by convention, use the first axis for the different spatial frequency axes (k_x, k_y, k_z).

For any bart tool which needs a trajectory, we need k-space shape to start with a singleton dimension for this reason - that's the reason for the reshape step in the beginning.

Next, we process the trajectory in the same manner as the k-space beforehand to generate the 'sliding window' trajectory:

✏️ **Re-Use the code from before to create a trajectory matching the sliding window data**:

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>
    
    %%bash
    ## create sliding window traj.
    for i in $(seq 4); do
        bart extract 10 $i 100 trj trj_$i
        bart resize 10 100 trj_$i trj_$i
    done
    bart join 11 trj $(seq -s' ' -f 'trj_%g' 4) tmp
    bart transpose 2 11 tmp tmp
    bart reshape $(bart bitmask 2 11) 65 1 tmp trj_sw
</details>

In [ ]:
!bart show -m trj_sw
!bart show -m ksp_sw

✅ You should now have two files:
- ksp_sw with dimensions (1, 256, 13, 1, ..., 1, 100) and
- trj_sw with dimensions (3, 256, 13, 1, ..., 1, 100)

## Ram-Lak filter and reconstruction

With trajectory in place, we can construct the Ram-Lak filter and multiply it with the k-space.

For multiplication (and accumulation), `bart fmac` can be used.

✏️ **k-Space filtering:**
- Use bart rss to create a Ram-Lak filter
- Create a filtered k-space by multiplying the filter to it

In [ ]:
%%bash


✏️ **Use the adjoint NuFFT to reconstruct the filtered k-space.**

In [ ]:
!bart nufft

The resulting image still has 6 channels - using a root-sum-of-squares coil combination, we can create a single image / movie:

✏️ Combine the coils of the output image using root-sum-of-squares:

In [ ]:
!bart rss 

<details>
    <summary>Show solutions</summary>

    %%bash
    
    bart rss 1 trj_sw filter
    bart fmac filter ksp_sw ksp_f
    bart nufft -a trj_sw ksp_f coil_img
    bart rss 8 coil_img img

</details>

✅ Have a look at the resulting image using view:

In [ ]:
!bart view img

# Part 2: Iterative reconstruction

In the last part, a simple, non-iterative algorithm was used to reconstruct a real-time dataset.

This part focusses on a step-wise improval of the algorithm.

## Inverse NuFFT

As a first step, replace adjoint NuFFT + Ram-Lak filter by an actual inverse NuFFT.
In principle, this is already an "iterative reconstruction"!

This also means it will be slow. To get some results quickly, try it first on the first 20 frames!

✏️ **inverse NuFFT reco**
- Extract the first 20 frames from sliding window ksp and trajectory
- Run an inverse NuFFT and look at te results

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>
    
    %%bash
    bart extract 10 0 20 ksp_sw ksp_small
    bart extract 10 0 20 trj_sw trj_small
    bart nufft -i trj_small ksp_small coil_img_inv
</details>

In [ ]:
!view coil_img_inv

What happened?! The image looks actually worse than before.

The artefacts are characteristic for a NuFFT on a too-small field of view.

The answer is in the `-o2` option we used for creating the trajectory earlier.
From the $\Delta k$ in the trajectory and the number of samples, the NuFFT can infer the original resolution.
However, as is not unusual for radial cardiac MRI, there are objects outside the selected FoV, which would lead to aliasing.

Due to oversampling and radial sampling, we actually acquired with a twice as large FoV, but the NuFFT doesn't know about it.

By multiplying the trajectory with the desired "overgridding" factor, we can reconstruct a larger FoV. `1.5` is a typical value.
We don't just use "2" because that would reconstruct a lot of empty air.

The problem did not occur with the previous reco, because we used the adjoint NuFFT, which never needs to estimate an image size (the normal NuFFT does not need to estimate any sizes either, because they are given by the input data shape).

✏️ **inverse NuFFT reco with overgridding**

- scale the trajectory so as to reconstruct on an 1.5oversampled grid
- re-run the iNuFFT
- Crop the resulting image, to get the same FoV as in the adjoint NuFFT Reco (`bart resize -c`)
- Use `bart rss` to combine the channels

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>
    
    %%bash
    bart scale 1.5 trj_small trj_small_os
    bart nufft -i trj_small_os ksp_small coil_img_inv_os
    bart resize -c 0 128 1 128 coil_img_inv_os coil_img_inv
    bart rss 8 coil_img_inv img_inv

</details>

✅ Compare both images using view:

What do you see? Does it match your expecations?

In [ ]:
!view img_inv img

### Bonus Exercise: iNuFFT without sliding window

Maybe you are a bit disappointed by the "washy" appearance of the iNuFFT result.

One probable cause is "data inconsistency": Data within a single sliding window k-space frame actually represents several successive timepoints, and there was motion between this timepoints.
Thus, there is no 'correct' image which the NuFFT could find that describes the data.

What happens if you 'get rid' of the sliding window already at this point?
Is inverse NuFFT / "iterative reco", but without PI/CS sufficient for RT-MRI?

Repeat the previous steps, but use the original (coil-compressed) k-space and traj instead of the sliding window variant:

✏️ inverse NuFFT reco without sliding window

- Extract the first 20 frames from the Non-sliding window ksp (ksp_cc) and trajectory (trj)
- scale the trajectory
- re-run the iNuFFT
- Crop the resulting image
- combine the channels
- Compare!

In [ ]:
%%bash


<details>
    <summary>Show solution</summary>
    
    %%bash
    
    bart extract 10 0 20 ksp_cc ksp_small2
    bart extract 10 0 20 trj trj_small2
    
    
    bart scale 1.5 trj_small2 trj_small_os2
    bart nufft -i trj_small_os2 ksp_small2 coil_img_inv_os2
    bart resize -c 0 128 1 128 coil_img_inv_os2 coil_img_inv2
    bart rss 8 coil_img_inv2 img_inv2

</details>


In [ ]:
!view img_inv img_inv2

## PI/CS

### Sensitivities

A big factor we haven't used so far is parallel imaging.

However, to use parallel imaging, you have to estimate coil sensitivities.

This is done in the next step using a bart command named "ncalib". We will later see how it works;

In [ ]:
%%bash

bart extract 10 0 5 ksp_cc ksp_f0
bart reshape $(bart bitmask 2 10) 65 1 ksp_f0 ksp_f0
bart reshape $(bart bitmask 2 10) 65 1 trj_turns trj_f0

bart scale 1.5 trj_f0 trj_f0

bart ncalib -t trj_f0 ksp_f0 coils

### Reco

Equipped with the coil sensitivities, which should now be stored in a file called coils(.cfl/hdr), classical parallel imaging becomes possible:

✏️ **Parallel Imaging**
- BART provides the pics tool. Run a simple pics reconstruction.

Note: What applied for the iNUFFT applies for pics as well: The trajectory needs to be scaled, and the image cropped afterwards.

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>
    
    bart pics -t trj_small_os ksp_small coils img_pics_os
    bart resize -c 0 128 1 128 img_pics_os img_pics
</details>

In [ ]:
!view img_pics

✅ Compare with the adjoint NuFFT result.

The result should look slightly better.


### Bonus: PICS without sliding window

Is parallel imaging / compressed sensing enough to do RT-MRI?

✏️ **Try the above pics command with the non-sliding window dataset**

If you skipped the previous bonus exercise, you can extract the first 20 frames from traj. and k-space first, and rescale the trajectory, with the following commands:

In [ ]:
%%bash
bart extract 10 0 20 ksp_cc ksp_small2
bart extract 10 0 20 trj trj_small2
bart scale 1.5 trj_small2 trj_small_os2

---

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>

    bart pics -t trj_small_os2 ksp_small2 coils img_pics_os2
    bart resize -c 0 128 1 128 img_pics_os2 img_pics2
</details>

In [ ]:
!view img_pics2 img

### Bonus 2: Add Regularization

This is intended to be completed after the iGRASP part is done: The idea is to use the TV Regularization parameter on this dataset.

In [ ]:
%%bash

<details>
    <summary>Show solution</summary>
    
    %%bash
    bart pics -RT:1024:0:0.01 -t trj_small_os2 ksp_small2 coils img_pics_os3
    bart resize -c 0 128 1 128 img_pics_os3 img_pics3
</details>

In [ ]:
!view img_pics3